In [ ]:
## This file implements neural networks and logistic regression on p003ppresabs_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## To deal with overfitting problem, we include both dropout and regularizer when set up layers in neural networks.
## For fully-connected neural networks, the accuracy is 99.25% for combination data, and 98.61% for over-sampling data.
## For logistic regression, the accuracy is 100% for combination data, and 97.92% for over-sampling data.
## Since the accuracy scores are pretty high in logistic regression, we further construct random forest models, which 
## are relatively less likely to bring overfitting compared to decision tree.
## For random forest, the accuracy is 99.25% for combination data, and 100% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 98.41% for combination data, and 97.63% for over-sampling.

In [36]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p003ppresabs_quant.csv')
df.shape

(255, 866)

In [37]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [38]:
df['pheno']

0      0.169500
1      0.316500
2      0.460000
3      0.169000
4      0.171333
5      0.236333
6      0.166500
7      0.173333
8      0.315833
9      0.162167
10     0.186167
11     0.382833
12     0.170667
13     0.173333
14     0.162833
15     0.164333
16     0.382667
17     0.397000
18     0.372667
19     0.166333
20     0.242333
21     0.305833
22     0.162333
23     0.165500
24     0.616833
25     0.170167
26     0.491500
27     0.167833
28     0.166500
29     0.374833
         ...   
225    0.267200
226    0.293800
227    0.383400
228    0.513500
229    0.507700
230    0.311500
231    0.236000
232    0.228833
233    0.219333
234    0.167333
235    0.265167
236    0.160500
237    0.159333
238    0.165500
239    0.327833
240    0.342000
241    0.308500
242    0.170333
243    0.165333
244    0.340833
245    0.162000
246    0.170500
247    0.164500
248    0.164000
249    0.180000
250    0.348000
251    0.232500
252    0.166167
253    0.276667
254    0.225333
Name: pheno, Length: 255

In [39]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [40]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [41]:
df['pheno']
df['pheno'].value_counts()

0    240
1     15
Name: pheno, dtype: int64

In [42]:
df.shape

(255, 866)

In [43]:
df_clean = df.drop(columns=['id'])

In [44]:
df_clean.shape

(255, 865)

In [45]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 864) (255,)


In [46]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 209), (1, 235)]


In [47]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 240), (1, 240)]


In [48]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [49]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [ ]:
############# Fully-Connected Neural Network ################

In [50]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [51]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [52]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [53]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 310 samples, validate on 134 samples
Epoch 1/100
310/310 [==============================] - 1s 2ms/step - loss: 2.1263 - accuracy: 0.7065 - val_loss: 0.6879 - val_accuracy: 0.8582
Epoch 2/100
310/310 [==============================] - 0s 145us/step - loss: 1.9910 - accuracy: 0.7935 - val_loss: 0.6045 - val_accuracy: 0.8955
Epoch 3/100
310/310 [==============================] - 0s 235us/step - loss: 2.0103 - accuracy: 0.7839 - val_loss: 0.5700 - val_accuracy: 0.9030
Epoch 4/100
310/310 [==============================] - 0s 192us/step - loss: 2.1195 - accuracy: 0.7903 - val_loss: 0.5428 - val_accuracy: 0.8955
Epoch 5/100
310/310 [==============================] - 0s 344us/step - loss: 1.8438 - accuracy: 0.8000 - val_loss: 0.5307 - val_accuracy: 0.9328
Epoch 6/100
310/310 [==============================] - 0s 177us/step - loss: 2.3688 - accuracy: 0.7710 - val_loss: 0.5120 - val_accuracy: 0.9403
Epoch 7/100
310/310 [==============================] - 0s 211us/step - loss: 1.9734 - 

310/310 [==============================] - 0s 183us/step - loss: 1.6858 - accuracy: 0.9000 - val_loss: 0.2542 - val_accuracy: 0.9925
Epoch 57/100
310/310 [==============================] - 0s 127us/step - loss: 1.4336 - accuracy: 0.9161 - val_loss: 0.2532 - val_accuracy: 0.9925
Epoch 58/100
310/310 [==============================] - 0s 129us/step - loss: 1.6303 - accuracy: 0.9032 - val_loss: 0.2594 - val_accuracy: 0.9925
Epoch 59/100
310/310 [==============================] - 0s 154us/step - loss: 1.9282 - accuracy: 0.8839 - val_loss: 0.2614 - val_accuracy: 0.9851
Epoch 60/100
310/310 [==============================] - 0s 172us/step - loss: 1.5269 - accuracy: 0.9097 - val_loss: 0.2581 - val_accuracy: 0.9851
Epoch 61/100
310/310 [==============================] - 0s 151us/step - loss: 2.1747 - accuracy: 0.8677 - val_loss: 0.2551 - val_accuracy: 0.9925
Epoch 62/100
310/310 [==============================] - 0s 120us/step - loss: 2.0208 - accuracy: 0.8774 - val_loss: 0.2497 - val_accuracy

In [54]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

134/134 [==============================] - 0s 239us/step
combination test accuracy: 99.25%


In [55]:
###### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [56]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [57]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 336 samples, validate on 144 samples
Epoch 1/100
336/336 [==============================] - 1s 2ms/step - loss: 2.1825 - accuracy: 0.6726 - val_loss: 0.6722 - val_accuracy: 0.7500
Epoch 2/100
336/336 [==============================] - 0s 167us/step - loss: 2.0882 - accuracy: 0.7024 - val_loss: 0.6222 - val_accuracy: 0.7917
Epoch 3/100
336/336 [==============================] - 0s 407us/step - loss: 1.7454 - accuracy: 0.7500 - val_loss: 0.6186 - val_accuracy: 0.7222
Epoch 4/100
336/336 [==============================] - ETA: 0s - loss: 1.4456 - accuracy: 0.84 - 0s 122us/step - loss: 2.5586 - accuracy: 0.7500 - val_loss: 0.5840 - val_accuracy: 0.8333
Epoch 5/100
336/336 [==============================] - 0s 226us/step - loss: 2.0931 - accuracy: 0.7768 - val_loss: 0.5533 - val_accuracy: 0.8333
Epoch 6/100
336/336 [==============================] - 0s 113us/step - loss: 2.2950 - accuracy: 0.7738 - val_loss: 0.5396 - val_accuracy: 0.8611
Epoch 7/100
336/336 [=======================

336/336 [==============================] - 0s 114us/step - loss: 1.6128 - accuracy: 0.8958 - val_loss: 0.3056 - val_accuracy: 0.9861
Epoch 57/100
336/336 [==============================] - 0s 113us/step - loss: 1.6651 - accuracy: 0.8869 - val_loss: 0.3142 - val_accuracy: 0.9861
Epoch 58/100
336/336 [==============================] - 0s 111us/step - loss: 1.6629 - accuracy: 0.8869 - val_loss: 0.3425 - val_accuracy: 0.9653
Epoch 59/100
336/336 [==============================] - 0s 107us/step - loss: 1.9126 - accuracy: 0.8690 - val_loss: 0.3143 - val_accuracy: 0.9861
Epoch 60/100
336/336 [==============================] - 0s 95us/step - loss: 1.5705 - accuracy: 0.8958 - val_loss: 0.2978 - val_accuracy: 0.9861
Epoch 61/100
336/336 [==============================] - 0s 95us/step - loss: 1.4443 - accuracy: 0.9018 - val_loss: 0.3013 - val_accuracy: 0.9861
Epoch 62/100
336/336 [==============================] - 0s 113us/step - loss: 1.4403 - accuracy: 0.8958 - val_loss: 0.3035 - val_accuracy: 

In [58]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

144/144 [==============================] - 0s 107us/step
over-sampling test accuracy: 98.61%


In [ ]:
############ Logistic Regression ############

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [25]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 100.00%


In [27]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 97.92%


In [100]:
############## Random Forest ##############

[1.        1.        0.9787234 1.        1.       ]
0.9957446808510639
0.008510638297872353


In [29]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [30]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 99.25%


In [31]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [32]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 100.00%


In [33]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [34]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.92063492 1.         1.         1.         1.        ]
0.9841269841269842


In [35]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.97058824 0.95588235 0.98529412 1.         0.96969697]
0.9762923351158646
